# CSE 204 Lab 7: Building a Neural Network From Scratch (Cont.)

J.B. Scoggins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jbscoggi/teaching/blob/master/Polytechnique/CSE204/Lab7.ipynb) 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jbscoggi/teaching/master?filepath=Polytechnique%2FCSE204%2FLab7.ipynb)

## Introduction

In this lab you will learn to create a simple multilayer perceptron (MLP) neural network from scratch using only the Numpy package for matrix-vector operations.  In order to train the network on several datasets, you will need to implement the back-propagation algorithm with gradient descent.  Before getting started, let's review the notation we will use in this lab and recall the stochastic gradient descent algorithm.

### Notation

We will consider simple feed-forward networks that can be described by the following recursive relationship,

$$
\begin{align}
&z^l = a^{l-1} W^l + b^l,\\
&a^l = \sigma^l(z^l),
\end{align}
$$

where $a^l$ is the output (activation) of layer $l$ which is a nonlinear function $\sigma^l$ of a linear transformation of the previous layer's output.  The linear transformation is performed using the weight matrix $W^l$ and bias vector $b^l$ associated with the layer $l$.  We will denote the last layer in the network with a capital $L$ superscript.  The recursion is stopped by setting $a^0 = x$, where $x$ is the input vector to our network.  Note that in the recursive expressions above, we implicitly assume that our input/output vectors are row vectors.  The reason for this will be evident later.

Taking this notation into account, we see that a network with $L-1$ hidden layers is fully expressed by its $L$ weight matrices, bias vectors, and activation functions.  We can denote the set of trainable parameters in our network by $\theta = \{ W^1, \dots, W^L, b^1, \dots, b^L \}$.

In this lab, we are only concerned with supervised learning tasks.  Recall that in supervised learning, we have a dataset represented by a list of $(x, y)$ pairs where $x$ is the input to our model and $y$ is the desired output.

- For regression problems where we want to fit a function $y = f(x)$, $x$ is the independent variable vector, and $y$ is the function value.
- In classification problems, $x$ will correspond to a set of attributes and $y$ the corresponding label.

The goal of supervised learning is to "train" our network by adjusting its parameters in order to minimize a cost function over the entire training set,

$$
\min_\theta \mathcal{L} = \min_\theta \frac{1}{N} \sum_{p=1}^N \ell_p 
$$

where $\ell_p$ is just a short-hand notation for $\ell_p = \ell(a^L(x_p), y_p)$ and $\ell(\hat{y}, y)$ denotes the particular form of the loss function being considered.  In this lab, we will use 2 different loss functions:

1. Quadratic Loss: $\ell(\hat{y}, y) = \|\hat{y} - y\|^2 / 2$
2. Cross-entropy Loss: $\ell(\hat{y}, y) = -\sum_j[ y_j \ln\hat{y}_j + (1-y_j)\ln(1-\hat{y}_j)]$


## Step 1: Build an untrainable network

Understanding (and implementing) the back-propagation algorithm can seem a little daunting at first.  Therefore, let's start by building out the functions we need just to create a network that cannot be trained.  First load the libraries.  _You should only have to do this once._

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Exercise 1: Implement the activation functions.

1. The easiest activation we can implement is the identity function whic simply returns the input as itself.  Implement this below in the class template `Identity`.  The `prime` function should implement the derivative of the activation.
2. The threshold activation function takes an input and returns 1 if the input is positive, otherwise 0.  Implement the `Threshold` class below. (hint: use the `np.where` function)
3. Recall that the sigmoid function is given as $\sigma(x) = 1 / (1 + \exp(-x))$.  Derive the derivative of $\sigma(x)$ and show that it can be represented as $\sigma'(x) = \sigma(x) [1 - \sigma(x)]$
4. Implement the `Sigmoid` class below.

In [ ]:
# Activation functions
class Identity:

    @staticmethod
    def __call__(x):
        return x

    @staticmethod
    def prime(x):
        return np.ones_like(x)

class Threshold:

    @staticmethod
    def __call__(x):
        return np.where(x > 0.0, 1.0, 0.0)
    
    @staticmethod
    def prime(x):
        return np.zeros_like(x)

    
class Sigmoid:

    @staticmethod
    def __call__(x):
        return 1.0 / (1.0 + np.exp(-x))

    @staticmethod
    def prime(x):
        sigmoid = Sigmoid()(x)
        return sigmoid * (1.0 - sigmoid)

4. Plot the activastion functions and their derivative on the domain [-5, 5].

In [ ]:
x = np.arange(-5, 5, 0.01)
plt.plot(x, Sigmoid()(x))
plt.plot(x, Sigmoid().prime(x))

During this lab, we will build on the following python class called `Network` which will represent our neural net.  The `Network` class will keep track of the weights, biases, and activations needed to evaluate and train our model.  The following exercises will guide you through building up the class from the skeleton below.

In [ ]:
class Network:
    """A simple implementation of a feed-forward artificial neural network.
    
    Work on this class throughout the entire lab, rerunning this cell after each update.
    """

    def __init__(self, sizes, activations):
        """Construct a network given a list of the number of neurons in each layer and 
        a list of activations which will be applied to each layer.
        
        :param sizes: A list of integers representing the number of nodes in each layer, 
        including the input and output layers.
        :param activations: A list of callable objects representing the activation functions.
        Its size should be one less than sizes.
        """
        
        self.sizes = sizes
        self.sigmas = activations
        
        self.biases  = [np.random.randn(1, n) for n in sizes[1:]]
        self.weights = [
            np.random.randn(m, n)/np.sqrt(m) for m, n in zip(sizes[:-1], sizes[1:])
        ]
    
    def num_params(self):
        """Returns the total number of trainable parameters in this network."""
        return sum([m*n + n for m,n in zip(self.sizes[:-1], self.sizes[1:])])
    
    def feed_forward(self, x):
        """Evaluates the network for the given input and returns the result.
        
        :param x: A numpy 2D-array where the columns represent input variables
        and rows represent independent samples.
        """
        a = x
        for W, b, sigma in zip(self.weights, self.biases, self.sigmas):
            a = sigma(np.matmul(a, W) + b)
        return a
        
    
    def backprop(self, x, y, cost):
        """Compute gradients of cost function w.r.t network parameters for a training point."""
        # Exercise 6.3
        pass

    def update_step(self, training_data, cost, learning_rate):
        """Compute the average parameter gradients over the whole training set and do a GD step.
        
        :param training_data: An (x, y) tuple containing the training data.
        :param cost: A callable object with signature cost(y_pred, y) for computing the cost of a single
        training example.
        :param learning_rate: The learning rate to use in the GD step.
        """
        # Exercise 6.4
        pass
    
    def train(self, training_data, cost, learning_rate, epochs):
        """Trains this network on the training data
        
        :param training_data: An (x, y) tuple containing the training data.
        :param cost: A callable object with signature cost(y_pred, y) for computing the cost of a single
        training example.
        :param learning_rate: The learning rate to use in the GD step.
        :param epochs: The number of epochs to train with.
        """
        for i in range(epochs):
            self.update_step(training_data, cost, learning_rate)
            
            # Compute the loss
            x, y = training_data
            loss = cost(self.feed_forward(x), y) / x.shape[0]
            print('epoch: ', i, ', loss: ', loss)

### Exercise 2: First draft of the Network class

1. From the recursion formulas above, write down the shapes of the weight matrices and bias vectors, given the number of neurons in each layer.
2. Implement the init `__init__` function in `Network` above.  Construct a list of weight matrices and bias vectors which are randomly initialized with normal distributions (see `np.random.randn`).
2. Implement the `num_params` function in `Network` above.  Use your knowledge of the shapes of the weights and biases.
3. Implement the `feed_forward` function in `Network` above. Use the recurance relations discussed in the Notation section.

### Exercise 3: Test feed-forward Network initialization

1. Create a network with 2 hidden layers of 4 nodes that takes 4 inputs and has 1 output. (Use any activation)

In [ ]:
network = Network( [4, 5, 5, 1], 3*[Sigmoid()] )

2. Compute by hand the number of parameters this network should have. (61)
3. Check that `num_params` provides the same value.

In [ ]:
print(network.num_params())

4. Print the weights and biases of the network and confirm they are intialized correctly (shape and values).

In [ ]:
print(network.weights, network.biases)

### Exercise 4: Build logic gates

Below is a table of logical functions (logic gates).  Each function takes two values (A and B) representing True (1) or False (0) propositions and returns a True or False value.

| A | B | AND | OR | XOR | NAND |
|---|---|-----|----|-----|------|
| 0 | 0 | 0   | 0  | 0   | 1    |
| 0 | 1 | 0   | 1  | 1   | 1    |
| 1 | 0 | 0   | 1  | 1   | 1    |
| 1 | 1 | 1   | 1  | 0   | 0    |

Interestingly, [it is possible to create any boolean function of any size through a combination of NAND gates](https://en.wikipedia.org/wiki/NAND_gate)!  Thus, if we can create a network which reproduces the logic behind a NAND-gate, it is possible to represent any logical function (and by extension any mathematical function) by combining such a network into ever more complex networks.  This is one version of a universal approximation theorem for ANNs.

**Task:** build the AND, OR, and NAND logic gates above using the simple network below by modifying its weights and biases directly.  I have provided the `logic_gate` function to help test your networks.


In [ ]:
def logic_gate(network):
    """Helper function for testing our network as a logic gate."""
    return network.feed_forward(
        np.asarray(
            [[0, 0],
            [0, 1],
            [1, 0],
            [1, 1]]
        )
    )

# Create a simple "logic gate" network
network = Network([2,1], [Threshold()])

# Modify the weights and biases to create the logic gates
# AND
network.weights[0] = np.asarray([[1], [1]])
network.biases[0] = np.asarray([-1.5])
print('AND(A,B) = \n', logic_gate(network))

# OR
network.weights[0] = np.asarray([[1], [1]])
network.biases[0] = np.asarray([-0.5])
print('OR(A,B) = \n', logic_gate(network))

# NAND
network.weights[0] = np.asarray([[-2], [-2]])
network.biases[0] = np.asarray([3])
print('NAND(A,B) = \n', logic_gate(network))



## Step 2: Training the network

While interesting, the `Network` class above is pretty useless as it stands since there is no way to learn a function we don't already know.  In this step, we will add the ability to train our network on a dataset using the gradient descent and back-propagation algorithms.  Let's review both of these algorithms now.

### (Stochastic) Gradient Descent

Recall from the beginning of the lab that we want to train the network parameters by minimizing a given loss function over an entire dataset.  One method of doing this is using the gradient descent (GD) algorithm which you have already seen this in the lecture portion of the class, thus I will just provide the update rule here.

$$
\theta = \theta - \eta \nabla_\theta \mathcal{L},
$$

where $\eta$ is the learning rate.  The update rule above is called GD because direction of change in the network parameters follow the opposite of the parameters' gradient in the loss function.  You can think of this like a ball rolling down a hill to find the minimum of the topology.  Only in this case, the ball is massless because it has no momentum.  When this update rule is applied to a random subset of the total dataset, it is called stochastic gradient descent (SGD).  SGD is far more efficient than GD when the batch size is large enough to approximate the true gradients while being significantly smaller than the full dataset.  Running over the entire dataset with SGD once is called an epoch of training.

### Back-Propagation

From the GD update rule above, it is clear we will need to compute the gradients of the network parameters with respect to the cost function.  This is exactly what back-propagation does, and thus is a crucial component to almost all neural network learning algorithms.  In the next exercise, we will derive the 4 equations in back-propagation.  You will need knowledge of the [chain rule for differentiation](https://en.wikipedia.org/wiki/Chain_rule) if you are not already familiar with this.

### Exercise 5: Derive backprop formulas

Before we start, it is convenient to define the following variable,

$$
\delta^l \equiv \frac{\partial \ell_p}{\partial z^l},
$$

Thus $\delta^l$ is the gradient of the loss function for a point $p$ with respect to the input to the activation function for the layer $l$ in our network.

1. What is the shape of $\delta^L$? (same as output)
2. Show that $\delta^L = \nabla_{a^L} \ell_p  \odot {\sigma'}^L ( z^L )$. Hint: apply the chain rule to the definition of $\delta^L$.
3. Show that for $l < L$, $\delta^l = [\delta^{l+1} (W^{l+1})^T ] \odot {\sigma'}^l ( z^l )$.  Hint: try computing $\delta^{L-1}$ and compare to $\delta^L$.
4. Show that $\nabla_{W^l}\ell_p = (a^{l-1})^T \delta^l$. Hint: use definition of $z^l$.
5. Show that $\nabla_{b^l}\ell_p = \delta^l$.

Note that these four formulas allow us to compute the gradient of the loss function for a single training point with respect to the parameters of our network.  Looking at the equations more closely, you should see a possible algorithm form.

1. Compute the values of $z^l$ and $a^l$ for $l = {1, \dots, L}$ by forward propagation through the network.  Recall $a^0 = x_p$.
2. Use the equation in Exercise 5.2 above to compute $\delta^L$.  
3. Back-propagate in the reverse direction using the equation in Exercise 5.3 to get all the other $\delta^l$ values.
4. Compute the parameter gradients using the equations in Exercises 5.4 and 5.5.

Note that this will yield the gradients for a single training point.  The gradients for the total loss function can easily be computed by

$$
\nabla_\theta \mathcal{L} = \frac{1}{N} \sum_{p=1}^N \nabla_\theta \ell_p.
$$

Of course, the exact form of $\nabla_\theta \ell_p$ will depend on the choice of $\ell$.

### Exercise 6: Implement the training algorithm

Before we can implement the training algorithm, we need to define a cost function.  For now, let's create a class representing the quadratic cost which was given in the introduction.

1. Show that the derivative of the quadratic cost is $\nabla_{\hat{y}} \ell(\hat{y}, y) = \hat{y} - y$.
2. Impement the quadratic cost function below.

In [ ]:
class QuadraticCost:

    @staticmethod
    def __call__(y_pred, y):
        """Computes the quadratic cost function."""
        return 0.5 * np.sum(np.square(y_pred - y))
    
    @staticmethod
    def derivative(y_pred, y):
        """Computes the derivative of the quadratic cost function."""
        return (y_pred - y)

With this cost defined, let's implement three remaining functions in `Network` that will allow us to train our models.  The first is the `backprop` method which takes a single training sample $x, y$ and a cost function object like the one above, and computes the gradient of the cost function using the backpropagation algorithm.

3. Implement the `backprop` funtion in the `Network` class.

Next, implement a single epoch of training by averaging the gradients over the whole dataset (calling `backprop` on each data point) and using the GD update rule to update the weights and biases.

4. Implement the `update_step` function in the `Network` class.

The `update_step` method is used by the `train` method successively in a loop as you can see in the `Network` class implementation above.  Assuming you have correctly implemented the `backprop` and `update_step` functions correctly, you should now be able to train the network on real training data.

### Exercise 7: Test training on simple regression problem.

Run the following code to test the network.  If all is correct, you should see the totall loss dropping and the network will learn the sine function.  

1. Play with different hyperparameters of the network and training such as the number of layers, number of nodes per layer, training rate, and number of training samples.  Can you improve on the training performance?

In [ ]:
x = np.random.uniform(-1, 1, (100, 1))
y = np.sin(np.pi*x)

network = Network([1, 100, 1], [Sigmoid(), Identity()])
network.train((x, y), QuadraticCost(), 0.1, 10000)

x_pred = np.arange(-1, 1, 0.01).reshape(200,1)
y_pred = network.feed_forward(x_pred)

plt.plot(x, y, 'k.')
plt.plot(x_pred, y_pred, 'k-')
plt.show()

## Step 3: Hand-written digit classification

Let's test our model on a more sophisticated dataset, namely the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). This dataset includes 70,000 images of hand-written digits 28x28 pixels (=784), split into a training set of 60,000 images and a test set of 10,000 images. Each image is labeled with a digit from 0 to 9.  

### Exercise 8: Download the MNIST dataset

1. Just run the following cell to download the MNIST database and prepare it.

In [ ]:
!git clone https://github.com/hsjeong5/MNIST-for-Numpy.git
!mv ./MNIST-for-Numpy/mnist.py ./mnist.py
import mnist
mnist.init()
x_train, y_train, x_test, y_test = mnist.load()

2. Next check that the shapes of the training and test data are what you expext.

3. Get a feel for the dataset by viewing some images.

In [ ]:
index = 0
plt.imshow(x_train[index,:].reshape(28, 28))
print(y_train[index])

### Exercise 9: Create a one-hot encoding of the labels

`y_train` and `y_test` currently contain the digits 0-9, which is not suited for training as we want to perform a 10-class classification training on our network. To fix this, we need to perform a so-called one-hot encoding on the labels which convert each digit into a vector of length 10 full of zeros, except the element corresponding to the digit which should be 1.  For example, if our label is 3, then the corresponding one-hot encoding is [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].

1. Convert the `y_train` and `y_test` vectors into one-hot encodings.

### Exercise 10: Train a network to recognize digits

1. Implement the cross-entropy loss function below.

In [ ]:
class CrossEntropy:

    @staticmethod
    def __call__(y_pred, y):
        pass
    
    @staticmethod
    def derivative(y_pred, y):
        pass

2. Train a model with 1 hidden layers of 30 nodes each, using sigmoidal activations and a cross-entropy loss.

In [ ]:
# Create the network

In [ ]:
# Train the network for 30 epochs with a learning rate of 0.5

3. Compute the accuracy of the model on the test data and compare a few predictions to their images.

## Extra

At this point you have a fully functioning neural network model which can be used in real machine-learning problems.  There are many things which you could add at this point to make the model more effective.  Consider playing with some of the following:

- Adding weight regularization
- Using different activation functions such as ReLU or Tanh
- Improve the efficiency by making the backprop algorithm work on whole batches at once
- Implement the SGD algorithm
- Redesign the model to allow plug-and-play layers to be stacked together, allowing more complicated layers such as convolutions to be used.

Of course the list is endless, just have fun!

## Next Step: Using Tensorflow

As you have now seen, building a neural network and training from scratch can require a significant amount of effort.  Furthermore, the naive design of our `Network` class does not allow us to easily incorporate new kinds of layers such as convolutions, dropout, or batch normalization.  In practice of course, several libraries already exist that provide all of this functionality for your, so that you can focus on building a model for your specific dataset.  Here, we will show you how to redo the digit classification example above, with less than 10 lines of code using the Tensorflow library.  You can [find this example](https://www.tensorflow.org/tutorials) directly on the Tensorflow webiste if you are interested.  Let's walk through the code below.

### Import Tensorflow and download the MNIST dataset

Tensorflow includes a high-level neural network architecture building API called Keras which makes it easy to create neural network models using "off-the-shelf" layers.  In addition, Keras also provides commonly used datasets, such as the MNIST dataset.  In the first line below, Tensorflow is imported.  Second, the `keras.datasets` API is used to dowload the full dataset into two groups, a training set and a testing set, like you have seen before.  In the third line, the pixel data (integers between 0-255) is converted to real values between 0.0-1.0, making it easier for our NN model to learn.

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Build the NN model

In the next two lines, the Keras API is used to create a simple NN model with the following layers:

- `Flatten` takes an N-dimensional array and reshapes it into a 1D array. In this case, the images are 28x28 pixels, which get converted to a 784 1D tensor.
- `Dense` is exactly like the type of implicit layers we coded in our `Network` class.  That is, they take some 1D input and apply a linear transformation before applying a nonlinear activation.  In this case we are using 512 nodes in the dense layer.
- `Dropout` is a little more advanced.  As you have seen in the lecture, a dropout layer sets some percentage of the inputs to the layer to zero.  During training, each time data is propagated through the dropout layer, a different random set of nodes is zeroed out.  This forces the network to make more generalizations about the data and reduces overfitting.  When training is done, the dropout layer allows all the nodes to pass through, but multiplies the result by the dropout factor such that the sum remains the same.  In this case, 20% of the nodes are randomly cancled out during training and the result during evaluation is multiplied by 0.8.  
- The final layer is a dense layer of 10 nodes corresponding to the one-hot encoding of the digit labels.  Note that a softmax activation function is used so that these nodes will represent propbabilities.

In the last line, the model is "compiled" with an optimizer (Adam), loss function (sparse categorical crossentropy), and a metrics, which is responsible for providing useful output during training.  In this case, the `accuracy` metric is used which computes the total number of training samples that were correctly categorized as trianing progresses.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train and evaluate the model

In the last step, we train the model.  This happens in the first line below, were we call the model's `fit` function, which takes the training data and a number of epochs.  The number of batches per epoch is handled by default.  In the second line, we evaluate the model on our testing data to see how well it does on data it has never seen before.  Note that we get a 98% accuracy with this simple model!

In [ ]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)